In [1]:
def read_content(file_path, start_row, end_row, start_column, end_column):
    content = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for i in range(start_row-1, end_row):
            row_content = lines[i].split()[start_column-1:end_column]
            # 计算满足条件的元素数量
            count_in_range = sum(849 <= int(element) <= 878 for element in row_content[:2])
            # 修改条件判断，确保有且仅有一个元素在849和878之间
            if count_in_range == 1:
                # 转换科学记数法为浮点数
                row_content = [float(element) if 'E' in element else element for element in row_content]
                content.append(row_content)
    return content

#生成dat文件
def generate_dat_file(pairscontent, plumed_dat, index):
    with open(plumed_dat, 'w') as file:
        file.write("cmap: CONTACTMAP ...\n")
        for i in range(len(pairscontent)):
            atoms = f"{pairscontent[i][0]},{pairscontent[i][1]}"  # 拼接原子信息
            ref = pairscontent[i][3]  # 参考值
            weight = 1/len(pairscontent)
            switch = f"{{Q R_0=0.01 BETA=50.0 LAMBDA=1.5 REF={ref}}}"
            line = f"   ATOMS{i+1}={atoms} SWITCH{i+1}={switch} WEIGHT{i+1}={weight}\n"
            file.write(line)
        file.write("\n   SUM\n...\n\n")
        file.write(f"PRINT ARG=cmap FILE=colvar{index}\n")

# Inputfile
itp_position="/hpc2hdd/home/jchen901/data/genesis/GENESIS_Tutorials-2022/Works/SMC_AICG2+/Kleisin_interaction_test/onlyPro_noDNAProGo_NOBC/1_setup/step3_output.itp"

# Exe_code
for i in range(1, 11):      # 生成50个文件plumed.dat
    pairscontent=read_content(itp_position, 5769, 10087,1,5)
    plumed_dat=generate_dat_file(pairscontent, f"plumed{i}.dat", i)